Q5. Find best writer and director combination or any other statistically relevant position(cinematographer, composer, etc.) based on worldwide gross.
- Dimitry Monday - first try to change every instance of movie code to movie title

In [1]:
import pandas as pd
import numpy as np
import requests as rq
import sqlite3
from sklearn.preprocessing import OneHotEncoder
from zipfile import ZipFile
import tarfile
import gzip

In [120]:
# IMDB Name Data    
with gzip.open("zippedData/imdb.name.basics.csv.gz", "rb") as f:
    imdb_name_basics = pd.read_csv(f)

with gzip.open("zippedData/imdb.title.akas.csv.gz", "rb") as f:
    imdb_title_akas = pd.read_csv(f)

with gzip.open("zippedData/imdb.title.basics.csv.gz", "rb") as f:
    imdb_title_basics = pd.read_csv(f)

with gzip.open("zippedData/imdb.title.crew.csv.gz", "rb") as f:
    imdb_title_crew = pd.read_csv(f)
    
with gzip.open("zippedData/imdb.title.principals.csv.gz", "rb") as f:
    imdb_title_principals = pd.read_csv(f)

with gzip.open("zippedData/imdb.title.ratings.csv.gz", "rb") as f:
    imdb_title_ratings = pd.read_csv(f)
    
# TN Movie Data
with gzip.open("zippedData/tn.movie_budgets.csv.gz", "rb") as f:
    tn_movie_budgets = pd.read_csv(f)

In [121]:
tn_movie_budgets.head(1)

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


In [122]:
imdb_title_basics.head(1)

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"


In [123]:
tn_movie_budgets = tn_movie_budgets.rename(columns={'movie': 'primary_title'})
tn_movie_budgets.head(1)

,id,release_date,primary_title,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"


In [124]:
movie_df = pd.merge(tn_movie_budgets, imdb_title_basics, on='primary_title')
movie_df.head(1)

,id,release_date,primary_title,production_budget,domestic_gross,worldwide_gross,tconst,original_title,start_year,runtime_minutes,genres
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279",tt1775309,Abatâ,2011,93.0,Horror


In [125]:
m_df = movie_df.drop('id', axis=1)
m_df = m_df.drop('release_date', axis=1)
m_df = m_df.drop('domestic_gross', axis=1)
m_df = m_df.drop('original_title', axis=1)
m_df = m_df.drop('runtime_minutes', axis=1)
m_df.head(1)

,primary_title,production_budget,worldwide_gross,tconst,start_year,genres
0,Avatar,"$425,000,000","$2,776,345,279",tt1775309,2011,Horror


In [127]:
imdb_df = imdb_movie_titles.drop('original_title', axis=1)
imdb_df = imdb_df.drop('start_year', axis=1)
imdb_df = imdb_df.drop('runtime_minutes', axis=1)
imdb_df = imdb_df.drop('genres', axis=1)
imdb_df = imdb_df.drop('primary_title', axis=1)
imdb_df.head(1)

,tconst,directors,writers
0,tt0063540,nm0712540,"nm0023551,nm1194313,nm0347899,nm1391276"


In [128]:
imdb_df1 = pd.merge(m_df, imdb_df, on='tconst')
imdb_df1.head(1)

,primary_title,production_budget,worldwide_gross,tconst,start_year,genres,directors,writers
0,Avatar,"$425,000,000","$2,776,345,279",tt1775309,2011,Horror,nm3786927,"nm2179863,nm4392664"


In [129]:
imdb_df1['worldwide_gross'] = imdb_df1['worldwide_gross'].str.replace(',', '')
imdb_df1['worldwide_gross'] = imdb_df1['worldwide_gross'].str.replace('$', '')
imdb_df2 = imdb_df1['worldwide_gross'].astype(float)
imdb_df1['worldwide_gross'] = imdb_df2
imdb_gross = imdb_df1.sort_values('worldwide_gross', ascending=False)

In [131]:
imdb_gross.head(1)

,primary_title,production_budget,worldwide_gross,tconst,start_year,genres,directors,writers
0,Avatar,"$425,000,000",2.776345e+09,tt1775309,2011,Horror,nm3786927,"nm2179863,nm4392664"


In [149]:
imdb_title_principals.head(1)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"


In [148]:
# create a database of directors
imdb_directors = imdb_title_principals.loc[imdb_title_principals['category'] == 'director']
imdb_directors.head(1)

,tconst,ordering,nconst,category,job,characters
1,tt0111414,2,nm0398271,director,NaN,NaN


In [145]:
imdb_name_basics.head(1)

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"


In [153]:
imdb_crew_directors = pd.merge(imdb_directors, imdb_name_basics, on='nconst')
imdb_crew_directors.tail(1)

,tconst,ordering,nconst,category,job,characters,primary_name,birth_year,death_year,primary_profession,known_for_titles
146392,tt9692684,3,nm10441594,director,NaN,NaN,Guy Jones,NaN,NaN,director,NaN


In [163]:
imdb_crew_directors1 = imdb_crew_directors.sort_values('primary_name')
imdb_crew_directors1.head(1)

,tconst,ordering,nconst,category,job,characters,primary_name,birth_year,death_year,primary_profession,known_for_titles
69576,tt3433162,5,nm2764095,director,NaN,NaN,'Atlas' Ramachandran,NaN,NaN,"actor,director","tt3433162,tt1311658,tt2333760,tt0928194"


In [164]:
imdb_crew_directors1 = imdb_crew_directors1.drop('ordering', axis=1)
imdb_crew_directors1 = imdb_crew_directors1.drop('characters', axis=1)
imdb_crew_directors2 = imdb_crew_directors1.drop('birth_year', axis=1)

In [165]:
imdb_crew_directors2.head(1)

,tconst,nconst,category,job,primary_name,death_year,primary_profession,known_for_titles
69576,tt3433162,nm2764095,director,NaN,'Atlas' Ramachandran,NaN,"actor,director","tt3433162,tt1311658,tt2333760,tt0928194"


In [198]:
imdb_name_gross = pd.merge(imdb_gross, imdb_crew_directors2, on='tconst')
imdb_name_gross.head(1)

,primary_title,production_budget,worldwide_gross,tconst,start_year,genres,directors,writers,nconst,category,job,primary_name,death_year,primary_profession,known_for_titles
0,Avatar,"$425,000,000",2.776345e+09,tt1775309,2011,Horror,nm3786927,"nm2179863,nm4392664",nm3786927,director,NaN,Atsushi Wada,NaN,director,"tt1594549,tt1775309,tt1602612"


In [187]:
imdb_name_gross.sort_values('directors', ascending=False).head(1)

,primary_title,production_budget,worldwide_gross,tconst,start_year,genres,directors,writers,nconst,category,job,primary_name,death_year,primary_profession,known_for_titles
1739,The Circle,"$18,000,000",40651864.0,tt8680408,2016,Drama,nm9961942,"nm9961944,nm9961943,nm9961942,nm9961946",nm9961942,director,NaN,Sarah R. Larson,NaN,"director,writer,editor","tt8856098,tt8680408"


In [199]:
imdb_name_gross1 = imdb_name_gross.drop('writers', axis=1)
imdb_name_gross1 = imdb_name_gross1.drop('primary_profession', axis=1)
imdb_name_gross1 = imdb_name_gross1.drop('category', axis=1)
imdb_name_gross1 = imdb_name_gross1.drop('job', axis=1)
imdb_name_gross1 = imdb_name_gross1.drop('tconst', axis=1)
imdb_name_gross2 = imdb_name_gross1.drop('known_for_titles', axis=1)

In [219]:
imdb_name_gross2['genres'].values.all()

'Drama,Horror,Thriller'

In [ ]:
imdb_name_gross2

In [233]:
m_df['genres'].value_counts

<bound method IndexOpsMixin.value_counts of 0                         Horror
1       Action,Adventure,Fantasy
2        Action,Adventure,Sci-Fi
3        Action,Adventure,Sci-Fi
4        Action,Adventure,Sci-Fi
                  ...           
3810                         NaN
3811                         NaN
3812                Comedy,Drama
3813      Horror,Sci-Fi,Thriller
3814       Drama,Horror,Thriller
Name: genres, Length: 3815, dtype: object>